In [1]:
import logging as log
import numpy as np
import random

In [2]:
log.basicConfig(format="%(messages)s",level=log.INFO)

In [ ]:
 ''' 加载数据，源数据为多行两列，每行的两列分别表示(X,Y) '''
def load_data(file_path):
    data = []
    with open(file_path, 'r', encoding = 'utf-8') as fr:
        for line in fr.read().splitlines():
            line_float = list(map(float, line.splite('\t')))
            data.append(line_float)
    data = np.array(data)
    return data 

In [3]:
'''计算两个点之间的欧氏距离'''
def score_euclidean(a, b):
    s = np.sqrt(np.sum(np.power(a, b)))
    return s 

In [4]:
'''随机采样k个样本并使其作为聚类中心'''
def rand_center(data, k):
    centers = np.array(random.sample(list(data),k))
    return centers

In [ ]:
def k_means(data, k, max_iter=100, score=score_euclidean, e=1e-6):
    """
    K-Means 算法
    一般 K-Mean 算法的终止条件有如下几个：
        1. 所有样本的类别不再改变
        2. 达到最大迭代次数
        3. 精度达到要求（？）
    返回聚类中心及聚类结果
    """
    # 样本数
    n = len(data)

    # 保存结果
    # 每个结果为一个二元组 [label, score] 分别保存每个样本所在的簇及距离质心的距离
    ret = np.array([[-1, np.inf]] * n)
        '''
    array([[-1., inf],
       [-1., inf],
       [-1., inf],
       [-1., inf],
       [-1., inf],
       [-1., inf],
       [-1., inf],
       [-1., inf],
       [-1., inf],
       [-1., inf]])
       
       '''
    # 初始化聚类中心（随机在数据中选取k条数据作为中心）
    centers = rand_center(data, k)

    changed = True  # 初始化样本类别是否改变(初始化为是)
    n_iter = 0  # 初始化迭代次数
    while changed and n_iter < max_iter:    #如果有样本类别的改变并且迭代次数为达到设定值
        changed = False                       #将changed先设为False并且迭代次数+1
        n_iter += 1

        for i in range(n):     # 对于每个训练数据
            i_score = np.inf    #初始化距离和标签
            i_label = -1
            for j in range(k):  # 分别与每个质心比较，k是质心个数
                s_ij = score_euclidean(data[i], centers[j])  #s_ij为当前循环的点到当前循环的质心的距离
                if s_ij < i_score:              #如果该距离小于当前该点到对应质心的距离
                    i_score = s_ij              #则替换成新的距离和分类
                    i_label = j

            if ret[i, 0] != i_label:  # 样本的类别发生了改变
                changed = True

            ret[i, :] = i_label, i_score        #将计算完成的类别和距离更新进第i条数据，行对应第i行，列对应所有列    

        # 记录更新后的聚类中心
        log.info(centers)           
        for i in range(k):
            data_i = data[ret[:, 0] == i]  # 标签为 i 的样本
            centers[i, :] = np.mean(data_i, axis=0)  # 按类别过滤样本

    log.info(n_iter)  # 迭代次数
    return centers, ret


In [ ]:
def test():
    file_path = r"./data.txt"
    data = load_data(file_path)
    print(data)
    print(np.shape(data)[1])
    
    s = score_euclidean(data[0], data[1])
    print(s)
    
    centers = rand_center(data, 3)
    print(centers)

In [ ]:
if __name__ == '__main__':
    file_path = "./data.txt"
    data = load_data(file_path)
    
    centers, ret = k_means(data, 3)